## FOR 50 JOBS

### IP ALGORITHM

In [1]:
import random

def generateData(machines=4, jobs=50, seed=7677):
    PROC = [ [ 0 for m in range(machines) ] for j in range(jobs) ]
    random.seed(seed)
    for j in range(jobs):
        for m in range(machines):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [2]:
import pulp

def IP(PROC):

    steps = [ len(job) for job in PROC ]
    assert(min(steps)==max(steps))
    N_MACHINES = len(PROC[0])
    N_JOBS = len(PROC)


    def val(x):
        return int(pulp.value(x))

    def proc(job, machine):
        return pulp.lpSum( [ PROC[j][machine] * JJ[job][j] for j in range(N_JOBS) ] ) 

    JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)), lowBound=0, upBound=1, cat='Integer')
    Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')

    prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)
    prob += Stop[N_JOBS-1][N_MACHINES-1]

    # JJ is a permutation of the jobs
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
        prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            prob += Start[j][m] + proc(j,m) == Stop[j][m]

    solvers = pulp.listSolvers(onlyAvailable=True) 
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
                
    return acc, int(pulp.value(prob.objective))

In [3]:
import numpy as np

def totalTime(PROC, seq, log=False):
    
    def isPermutation(seq):
        for i in range(len(seq)):
            if i not in seq:
                return False
        return True
    
    steps = [ len(job) for job in PROC ]
    assert(len(PROC) == len(seq))
    assert(isPermutation(seq))
    assert(min(steps)==max(steps))
    n_machines = len(PROC[0])
    n_jobs = len(PROC)
    wait = np.zeros([n_jobs, n_machines], dtype=int)
    idle = np.zeros([n_jobs, n_machines], dtype=int)
    start = np.zeros([n_jobs, n_machines], dtype=int)
    stop  = np.zeros([n_jobs, n_machines], dtype=int)
    proc = np.zeros([n_jobs, n_machines], dtype=int)
    for job in range(n_jobs):
        proc[job] = PROC[seq[job]]
    for job in range(n_jobs):
        for machine in range(n_machines):
            start[job, machine] = max(stop[job-1, machine] if job>0 else 0, stop[job, machine-1] if machine>0 else 0)
            wait[job, machine] = start[job, machine] - (stop[job, machine-1] if machine>0 else 0)
            idle[job, machine] = start[job, machine] - (stop[job-1, machine] if job>0 else 0)
            stop[job, machine] = start[job, machine] + proc[job, machine]

    if log:

        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)
        
        row = '|         |'
        for m in range(n_machines):
            row += f"            |" \
                    f" Machine: {m:1d} |"
        print(row)
        
        for j in range(n_jobs):
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Idle: {idle[j,m]:4d} |"
            print(row)
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Start:{start[j,m]:4d} |"  
            print(row)
            
            row = f'| Job: {seq[j]:2d} |'
            for m in range(n_machines):
                row += f" Wait: {wait[j,m]:4d} | " \
                       f"Proc: {proc[j,m]:4d} |"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Stop: {stop[j,m]:4d} |"  
            print(row)
            
        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)

    
    return stop[n_jobs-1, n_machines-1]                                                       

In [4]:
PROC = generateData(machines=4, jobs=50, seed=7677)

Compute the optimal job schedule:

In [5]:
seq, proctime = IP(PROC)
print(seq)
print(proctime)

[17, 1, 39, 4, 33, 21, 20, 48, 18, 27, 49, 31, 14, 15, 38, 41, 46, 23, 42, 37, 47, 16, 13, 0, 45, 9, 25, 35, 29, 5, 6, 8, 26, 2, 24, 44, 11, 10, 36, 22, 12, 7, 43, 28, 30, 34, 3, 19, 40, 32]
262


Check the processing time and print the schedule

In [6]:
totalTime(PROC, seq)

262

In [7]:
totalTime(PROC, seq, log=True)

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    5 |            | Idle:    6 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   5 |            | Start:   6 |
| Job: 17 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    8 |
|         |            | Stop:    1 |            | Stop:    5 |            | Stop:    6 |            | Stop:   14 |
|---------|------------|------------|------------|------------|---------

262

## Genetic Algorithm

In [17]:
import numpy as np
import random

def genetic_algorithm(PROC, population_size=100, generations=100, crossover_rate=0.8, mutation_rate=0.1):
    def create_individual():
        return random.sample(range(len(PROC)), len(PROC))

    def crossover(parent1, parent2):
        size = len(parent1)
        child = [-1] * size
        start, end = sorted(random.sample(range(size), 2))
        child[start:end+1] = parent1[start:end+1]
        pointer = 0
        for i in range(size):
            if child[i] == -1:
                while parent2[pointer] in child:
                    pointer += 1
                child[i] = parent2[pointer]
        return child

    def mutate(individual):
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]

    def selection(population):
        return random.choices(population, weights=[1/fitness(ind) for ind in population], k=2)

    def fitness(individual):
        return totalTime(PROC, individual)

    # Initial population
    population = [create_individual() for _ in range(population_size)]
    best_individual = min(population, key=fitness)
    best_fitness = fitness(best_individual)

    for generation in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = selection(population)
            if random.random() < crossover_rate:
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]
            if random.random() < mutation_rate:
                mutate(child1)
            if random.random() < mutation_rate:
                mutate(child2)
            new_population.extend([child1, child2])

        population = new_population
        current_best = min(population, key=fitness)
        current_best_fitness = fitness(current_best)

        if current_best_fitness < best_fitness:
            best_individual = current_best
            best_fitness = current_best_fitness

        print(f"Generation {generation+1}/{generations}: Best Time = {best_fitness}")

    return best_individual, best_fitness


In [18]:

# The genetic algorithm usage
best_seq, best_time = genetic_algorithm(PROC)

# Printing the best sequence and time found by the genetic algorithm
print(f"Best sequence: {best_seq}")
print(f"Best total time: {best_time}")

# detailed log of processing for the best sequence
totalTime(PROC, best_seq, log=True)


Generation 1/100: Best Time = 269
Generation 2/100: Best Time = 268
Generation 3/100: Best Time = 268
Generation 4/100: Best Time = 268
Generation 5/100: Best Time = 268
Generation 6/100: Best Time = 268
Generation 7/100: Best Time = 268
Generation 8/100: Best Time = 268
Generation 9/100: Best Time = 268
Generation 10/100: Best Time = 268
Generation 11/100: Best Time = 268
Generation 12/100: Best Time = 268
Generation 13/100: Best Time = 268
Generation 14/100: Best Time = 268
Generation 15/100: Best Time = 267
Generation 16/100: Best Time = 267
Generation 17/100: Best Time = 267
Generation 18/100: Best Time = 267
Generation 19/100: Best Time = 267
Generation 20/100: Best Time = 267
Generation 21/100: Best Time = 267
Generation 22/100: Best Time = 267
Generation 23/100: Best Time = 267
Generation 24/100: Best Time = 265
Generation 25/100: Best Time = 265
Generation 26/100: Best Time = 265
Generation 27/100: Best Time = 265
Generation 28/100: Best Time = 265
Generation 29/100: Best Time 

265

## Ant Colony Optimization

In [19]:
import random
import numpy as np

def AntColonyOptimization(PROC, n_ants=20, n_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5):
    N_JOBS = len(PROC)
    
    # Initialize pheromone levels
    pheromone = np.ones((N_JOBS, N_JOBS))
    best_seq = None
    best_time = float('inf')

    def calculate_probabilities(available_jobs, current_job):
        probabilities = []
        pheromones = []
        heuristics = []
        for job in available_jobs:
            pheromones.append(pheromone[current_job][job])
            heuristics.append(1.0 / PROC[job][0])  # Use job processing time on the first machine as a heuristic
        
        total_pheromone_heuristic = sum((p ** alpha) * (h ** beta) for p, h in zip(pheromones, heuristics))
        for p, h in zip(pheromones, heuristics):
            prob = ((p ** alpha) * (h ** beta)) / total_pheromone_heuristic
            probabilities.append(prob)
        return probabilities

    def update_pheromone(trail, time):
        for i in range(N_JOBS - 1):
            pheromone[trail[i]][trail[i + 1]] += 1.0 / time

    for iteration in range(n_iterations):
        all_trails = []
        all_times = []

        for ant in range(n_ants):
            trail = []
            available_jobs = list(range(N_JOBS))
            current_job = random.choice(available_jobs)
            trail.append(current_job)
            available_jobs.remove(current_job)

            while available_jobs:
                probabilities = calculate_probabilities(available_jobs, current_job)
                next_job = random.choices(available_jobs, probabilities)[0]
                trail.append(next_job)
                available_jobs.remove(next_job)
                current_job = next_job

            total_time = totalTime(PROC, trail)
            all_trails.append(trail)
            all_times.append(total_time)

            if total_time < best_time:
                best_time = total_time
                best_seq = trail

        # Evaporate pheromone
        pheromone *= (1.0 - evaporation_rate)

        # Update pheromone
        for trail, time in zip(all_trails, all_times):
            update_pheromone(trail, time)

    return best_seq, best_time


In [20]:
# Running the Ant Colony Optimization
seq, proctime = AntColonyOptimization(PROC)
print("Optimal job sequence:", seq)
print("Total processing time:", proctime)

Optimal job sequence: [17, 18, 41, 46, 20, 48, 31, 15, 39, 22, 33, 47, 16, 19, 35, 14, 37, 44, 4, 9, 49, 29, 38, 1, 13, 24, 7, 10, 3, 45, 5, 21, 40, 0, 27, 23, 2, 42, 6, 8, 25, 12, 26, 28, 34, 32, 36, 30, 43, 11]
Total processing time: 263


In [21]:
# Logging the detailed process
totalTime(PROC, seq, log=True)

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    5 |            | Idle:    6 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   5 |            | Start:   6 |
| Job: 17 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    8 |
|         |            | Stop:    1 |            | Stop:    5 |            | Stop:    6 |            | Stop:   14 |
|---------|------------|------------|------------|------------|---------

263